In [34]:
import numpy as np
import sys
import pandas as pd
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


In [4]:
spark = SparkSession.builder.appName("mean").getOrCreate()

In [5]:
sc = spark.sparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.info("pyspark script logger initialized")

In [6]:
dataset=pd.read_csv("CompleteDataset.csv")
# dataset = sqlContext.read.csv("CompleteDataset.csv")
LOGGER.info("read dataset")

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df=dataset
df.columns


Index(['Unnamed: 0', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'GK diving', 'GK handling', 'GK kicking',
       'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM', 'ID',
       'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB',
       'Preferred Positions', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       'RS', 'RW', 'RWB', 'ST'],
      dtype='object')

In [8]:
col_needed=['Name','Club','Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy','Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength','Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM',
       'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       'RS', 'RW', 'RWB', 'ST']
df=df[col_needed]
df.to_csv('out.csv', sep=',',header=False)
nocolumns=len(df.columns)
norows=len(df)
sparkdataset = sqlContext.read.csv("out.csv")
oldColumns = sparkdataset.schema.names

newColumns = ['Index','Name','Club','Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy','Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength','Result','Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM',
       'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       'RS', 'RW', 'RWB', 'ST',]
from functools import reduce
sparkdataset = reduce(lambda sparkdataset, idx: sparkdataset.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)-1), sparkdataset)
sparkdataset.printSchema()
sparkdataset.show()



root
 |-- Index: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Aggression: string (nullable = true)
 |-- Agility: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- Ball control: string (nullable = true)
 |-- Composure: string (nullable = true)
 |-- Crossing: string (nullable = true)
 |-- Curve: string (nullable = true)
 |-- Dribbling: string (nullable = true)
 |-- Finishing: string (nullable = true)
 |-- Free kick accuracy: string (nullable = true)
 |-- Heading accuracy: string (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Jumping: string (nullable = true)
 |-- Long passing: string (nullable = true)
 |-- Long shots: string (nullable = true)
 |-- Marking: string (nullable = true)
 |-- Penalties: string (nullable = true)
 |-- Positioning: string (nullable = true)
 |-- Reactions: string (nullable = true)
 |-- Short passing: string (nullable = true)
 

In [9]:
# from functools import reduce
from operator import add
from pyspark.sql.functions import col, lit

# rowMean  = (sum(col(x) for x in firstrow.columns[1:]) / n)
from functools import reduce
from operator import add
# from pyspark.sql.functions import col, lit

# n = lit(len(sparkdataset.columns) - 1.0)
# rowMean  = (reduce(add, (col(x) for x in sparkdataset.columns[1:])) / n).alias("mean")

# sparkdataset.select(rowMean)

# from pyspark.sql.functions import col

# marksColumns = [col('marks1'), col('marks2')]

# averageFunc = sum(x for x in marksColumns)/len(marksColumns)

# df.withColumn('Result(Avg)', averageFunc).show(truncate=False)

from pyspark.sql.functions import udf, array
from pyspark.sql.types import DoubleType
sparkdatasetcolumns=[ col('Aggression'), col('Agility'), col('Balance'), col('Ball control'),
       col('Composure'), col('Crossing'), col('Curve'), col('Dribbling'), col('Finishing'),
       col('Free kick accuracy'),col('Heading accuracy'), col('Interceptions'),
       col('Jumping'), col('Long passing'), col('Long shots'), col('Marking'), col('Penalties'),
       col('Positioning'), col('Reactions'), col('Short passing'), col('Shot power'),
       col('Sliding tackle'), col('Sprint speed'), col('Stamina'), col('Standing tackle'),
       col('Strength'),col('Result'),col('Vision'),col('Volleys'),col('CAM'),col('CB'),col('CDM'),col('CF'),col('CM'),
                     col('LAM'),col('LB'),col('LCB'),col('LCM'),col('LDM'),col('LF'),col('LM'),col('LS'),col('LW'),
                    col('LWB'),col('RAM'),col('RB'),col('RCB'),col('RCM'),col('RDM'),col('RF'),col('RM'),col('RS'),
                    col('RW')]
#'Result','Vision', 'Volleys', 'CAM', 'CB', 'CDM', 'CF', 'CM',
       #'LAM', 'LB', 'LCB', 'LCM', 'LDM', 'LF', 'LM', 'LS', 'LW', 'LWB', 'RAM', 'RB', 'RCB', 'RCM', 'RDM', 'RF', 'RM',
       #'RS', 'RW', 'RWB', 'ST'
averageFunc =( sum(x for x in sparkdatasetcolumns))/len(sparkdatasetcolumns)
newdataset=sparkdataset.withColumn('mean', averageFunc)
lol=newdataset.sort(newdataset.mean.desc())
lol.toPandas().to_csv('output.csv',index=False)
lol.show()
#newdataset.sort(sparkdataset.mean.desc()).collect()
#.sort(sparkdataset.Result(Avg).desc()).collect()

#sorted(sparkdataset.select("Result(Avg)").show())






# from functools import reduce
# from operator import add
# from pyspark.sql.functions import col, lit

# n = lit(len(sparkdataset.columns) - 1.0)
# rowMean  = (reduce(add, (col(x) for x in sparkdataset.columns[1:])) / n).alias("mean")

# sparkdataset.select(rowMean).show()

#sparkdataset.sort(sparkdataset.mean.desc()).collect()



# avg_cols = udf(lambda array: sum(array)/len(array), DoubleType())

# sparkdataset.withColumn("average", avg_cols(array('Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
#        'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
#        'Free kick accuracy','Heading accuracy', 'Interceptions',
#        'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
#        'Positioning', 'Reactions', 'Short passing', 'Shot power',
#        'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
#        'Strength'))).show()

+-----+-------------+-------------------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+------+-------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+
|Index|         Name|               Club|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|Heading accuracy|Interceptions|Jumping|Long passing|Long shots|Marking|Penalties|Positioning|Reactions|Short passing|Shot power|Sliding tackle|Sprint speed|Stamina|Standing tackle|Strength|Result|Vision|Volleys| CAM|  CB| CDM|  CF|  CM| LAM|  LB| LCB| LCM| LDM|  LF|  LM|  LS|  LW| LWB| RAM|  RB| RCB| RCM| RDM|  RF|  RM|  RS|  R

In [39]:
from pyspark.sql.functions import count, avg
lol2=newdataset.groupBy("Club").agg(avg("mean"),count("*"))
oldColumns=['Club', 'avg(mean)', 'count(1)']
newColumns=['Club', 'mean', 'count(1)']
lol2 = reduce(lambda lol2, idx: lol2.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)-1), lol2)
lol2=lol2.sort(lol2.mean.desc())
lol2.toPandas().to_csv('output2.csv',index=False)

In [24]:
dataframe=pd.read_csv("CompleteDataset.csv")
col_needed = ['Overall','Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control',
       'Composure', 'Crossing', 'Curve', 'Dribbling', 'Finishing',
       'Free kick accuracy', 'Heading accuracy', 'Interceptions',
       'Jumping', 'Long passing', 'Long shots', 'Marking', 'Penalties',
       'Positioning', 'Reactions', 'Short passing', 'Shot power',
       'Sliding tackle', 'Sprint speed', 'Stamina', 'Standing tackle',
       'Strength', 'Vision', 'Volleys', 'Preferred Positions']

recol_needed = ['Overall','Finishing', 'Shot power', 'Positioning', 'Dribbling', 'Long shots','Penalties', 'Volleys', 
                 'Acceleration', 'Agility','Sprint speed', 'Curve',
                
       'Free kick accuracy', 'Heading accuracy', 
       'Short passing', 'Long passing', 'Vision',
       'Strength', 'Stamina', 'Balance', 'Ball control','Composure','Jumping', 
       'Crossing','Reactions',
       'Aggression','Interceptions', 'Marking', 'Sliding tackle', 'Standing tackle','Preferred Positions']
dataframe = dataframe[recol_needed]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
dataframe['Preferred Positions'] = dataframe['Preferred Positions'].str.strip()
#remove Goalkeeper from dataframe

dataframe = dataframe[dataframe['Preferred Positions'] != 'GK']
positions = dataframe['Preferred Positions'].str.split().apply(lambda x: x[0]).unique()
df_fifa = dataframe.copy()
df_fifa.drop(df_fifa.index, inplace=True)

for position in positions:
    temp = dataframe[dataframe['Preferred Positions'].str.contains(position)]
    temp['Preferred Positions'] = position
    df_fifa = df_fifa.append(temp, ignore_index=True)

cols = [col for col in df_fifa.columns if col not in ['Preferred Positions']]

for i in cols:
    df_fifa[i] = df_fifa[i].apply(lambda x: eval(x) if isinstance(x,str) else x)

df_fifa_normalized = df_fifa.iloc[:,:-1].div(df_fifa.iloc[:,:-1].sum(axis=1), axis=0)
mapping = {'ST': 1, 'RW': 1, 'LW': 1, 'RM': 1, 'CM': 1, 'LM': 1, 'CAM': 1, 'CF': 1, 'CDM': 0, 'CB': 0, 'LB': 0, 'RB': 0, 'RWB': 0, 'LWB': 0}

df_fifa_normalized['Preferred Positions'] = df_fifa['Preferred Positions']

df_fifa_normalized = df_fifa_normalized.replace({'Preferred Positions': mapping})
clf = LogisticRegression()
x = df_fifa_normalized.iloc[:,:-1]
y = df_fifa_normalized.iloc[:,-1]


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
clf = Ridge(alpha=0.001)
clf.fit(x,y)
Feature_Coef_list = list(sorted(zip(recol_needed, abs(clf.coef_)),key=lambda x: -x[1]))
Feature_Coef_table = pd.DataFrame(np.array(Feature_Coef_list).reshape(-1,2), columns = ['Attributes', 'Coefficient'])
print("Important features for prediction of position")
Feature_Coef_table

Important features for prediction of position


,Attributes,Coefficient
0,Marking,10.7953178394
1,Finishing,10.1068096178
2,Overall,9.44092922628
3,Vision,6.89757009575
4,Short passing,5.34600535511
5,Crossing,5.25926075923
6,Sliding tackle,4.5907538452
7,Interceptions,4.48911924143
8,Positioning,4.30206150463
9,Heading accuracy,3.8117216428
